In [1]:
import numpy as np
import os
import sys
import tensorflow as tf

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/home/xs/tensorflow_test/sycu-tensorflow-train/research/object_detection")
from object_detection.utils import ops as utils_opsjiqi

from utils import label_map_util
from utils import visualization_utils as vis_util

In [2]:
model_file_path = "./ssd_mobilenet_v1_sycu_20210328/saved_model"
label_map_path = "./ssd_mobilenet_v1_sycu_20210328/sycu_label_map.pbtxt"
NUM_CLASSES = 4
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [3]:
import cv2
import time
frame = cv2.imread('./ball.png')

In [4]:
with tf.Session() as sess:
    
    saver=tf.train.import_meta_graph('./ssd_mobilenet_v1_sycu_20210328/model.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./ssd_mobilenet_v1_sycu_20210328/'))
    #saver.restore(sess=sess, save_path=args.save_path)  # 读取保存的模型
    detect_graph = tf.get_default_graph()
    # 打印查看签名signature的内容
    #print(detect_graph.signature_def)
    image_np_expanded = np.expand_dims(frame, axis=0)

    image_tensor = detect_graph.get_tensor_by_name('image_tensor:0')

    boxes = detect_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
    scores = detect_graph.get_tensor_by_name('detection_scores:0')
    classes = detect_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detect_graph.get_tensor_by_name('num_detections:0')
    (num_detections,boxes, scores, classes) = sess.run(
                [num_detections, boxes, scores, classes],
                feed_dict={image_tensor: image_np_expanded})
    #print(boxes[:5], scores[:5], classes[:5])
    vis_util.visualize_boxes_and_labels_on_image_array(
                frame, np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores), category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
 
    cv2.imshow('object detection', frame)
    k = cv2.waitKey(0)
    if k == ord('q'):
        cv2.destroyAllWindows()

    

INFO:tensorflow:Restoring parameters from ./ssd_mobilenet_v1_sycu_20210328/model.ckpt
